
# SVM

Zaprezentujemy tutaj różne sposoby użycia SVM do przeprowadzenia klasyfikacji. Zacznijmy oczywiście od wczytania 
zbioru danych i szybkiego preprocessingu danych na wzór poprzednich prezentacji


In [1]:
from sklearn import datasets

dataset = datasets.load_iris()
import numpy as np
import pandas as pd

size = len(dataset.data)

data = np.concatenate([dataset.data, np.reshape(dataset.target, (size,1))], axis = 1)
column_names = dataset.feature_names[:]
column_names.append('target')
df = pd.DataFrame(data, columns = column_names, copy = True)
df.head()

X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y_bin = df['target'] == 2
y = df['target']



Ważnym elementem preprocessingu jest skalowanie danych - bez którego SVM jest nieodpowiednio dobierane


In [2]:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)

X_scaled = scaler.transform(X)



W reszcie podział danych na zbiory treningowe i testowe

In [3]:

from sklearn.model_selection import train_test_split

X_train, X_test , y_train, y_test = train_test_split(X_scaled, y)
X_bin_train, X_bin_test , y_bin_train, y_bin_test = train_test_split(X_scaled, y_bin)



Na pierwsze starcie wykorzystajmy model SVR z liniowym jądrem, czyli najprostszą implementację bez 
kernel-tricku.

Wykorzystując wersję LinearSVC zapewniamy stosowanie technik optymalizacyjnych odpowiadających 
problemom liniowym, zamiast kwadratowym jak to w przypadku ogólnego SVC.

Dostepnych jest dla niego wiele opcji konfiguracyjnych

*class sklearn.svm.LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)*


In [4]:

from sklearn.svm import LinearSVC

classifier = LinearSVC(C=1, penalty='l2', loss='squared_hinge', dual=False)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.8421052631578947

In [5]:

from sklearn.svm import LinearSVC

classifier = LinearSVC(C=1, penalty='l2', loss='squared_hinge', dual=False)
classifier.fit(X_bin_train, y_bin_train)
y_pred = classifier.predict(X_bin_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_bin_test, y_pred)

0.9736842105263158


# SVC z jądrem wielomianowym

Aby wykorzystać kernel-trick należy użyć już implemetancji SVC z optymalizacją kwadratową

*class sklearn.svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)*


In [6]:

from sklearn.svm import SVC

classifier = SVC(kernel = 'poly', degree = 3, C = 1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.8947368421052632

In [7]:
classifier = SVC(kernel = 'poly', degree = 3, C = 1)
classifier.fit(X_bin_train, y_bin_train)
y_pred = classifier.predict(X_bin_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_bin_test, y_pred)

0.8947368421052632


# SVC z jądrem RBF

Bardzo analogicznie postępujemy chcąc wykrozystać jądro RBF

In [8]:

from sklearn.svm import SVC

classifier = SVC(kernel = 'rbf', C = 1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(y_pred)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


[1. 2. 2. 2. 2. 2. 0. 1. 2. 0. 2. 2. 2. 0. 0. 2. 1. 0. 2. 1. 1. 1. 1. 2.
 0. 1. 0. 1. 0. 1. 2. 1. 1. 1. 0. 0. 0. 1.]


0.9210526315789473

In [9]:
classifier = SVC(kernel = 'rbf', C = 1)
classifier.fit(X_bin_train, y_bin_train)
y_pred = classifier.predict(X_bin_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_bin_test, y_pred)

0.9736842105263158

# Regresja SVM (SVR)

Omówimy również w jaki sposób stosować możemy SVM od przeprowadzania regresji. 
Rozpocznijmy od zaczytania zbioru danych do prezentacji działania regresji


In [10]:

from sklearn import datasets

dataset3 = datasets.load_diabetes()


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

size = len(dataset3.data)

data3 = np.concatenate([dataset3.data, np.reshape(dataset3.target, (size,1))], axis = 1)
column_names = dataset3.feature_names[:]
column_names.append('target')
df3 = pd.DataFrame(data3, columns = column_names, copy = True)
df3.head()



,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0



Dokonajmy jeszcze podziału danych


In [11]:
X = df3.drop('target', axis = 1)
y = df3['target']
X.head()

from sklearn.preprocessing import StandardScaler

scalerX = StandardScaler()

scalerX.fit(X)

X_scaled = scalerX.transform(X)

from sklearn.model_selection import train_test_split

X_train, X_test , y_train, y_test = train_test_split(X_scaled, y)



Dostępne są nam dwie implementacje modelu SVR (podobnie jak przy SVC) w wersji programowania
liniowego oraz kwadratowego

## Regresja liniowa

Wykorzystuje klasę LinearSVR postaci

*class sklearn.svm.LinearSVR(epsilon=0.0, tol=0.0001, C=1.0, loss='epsilon_insensitive', fit_intercept=True, intercept_scaling=1.0, dual=True, verbose=0, random_state=None, max_iter=1000)*



In [12]:

from sklearn.svm import LinearSVR 
regressor = LinearSVR(epsilon = 0.5)
regressor.fit(X, y)
y_pred = regressor.predict(X_test)

from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, y_pred)

0.3771538409077724


## Regresja nieliniowa z jądrem

Aby użyć jądra oraz kernel-trick należy użyć klasy SVR postaci:

*class sklearn.svm.SVR(kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)*


In [13]:

from sklearn.svm import SVR 
regressor = SVR(epsilon = 0, kernel = 'rbf')
regressor.fit(X, y)
y_pred = regressor.predict(X_test)

from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, y_pred)

-2.220446049250313e-16